In [1]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import koreanize_matplotlib
%config InlineBackend.figure_format = 'retina'
from glob import glob
import seaborn as sns
import requests


# 인증키, url 설정
auth_key = '67456d5167776a6437305742434844'
url = f'http://openapi.seoul.go.kr:8088/{auth_key}/xml/CardSubwayStatsNew/1/999/20230131'

# requests로 response값 받기
response = requests.get(url)
response.status_code

# xml 읽어오기
# 하루치 결과만 불러올 수 있었음 -> 한달을 분석하려면 30번 정도 불러와야 함
df = pd.read_xml(response.text)
df

# 23년 1월 한달간 서울시 지하철 혼잡도 데이터 불러오기
first_day = '20230101'
last_day = '20230131'
jan = pd.date_range(start=first_day, end=last_day) # 1월 한달간 날짜 수집
jan = jan.astype(str) # datetime -> object 
jan = list(jan)
jan_list = [day.replace('-','') for day in jan]

auth_key = '67456d5167776a6437305742434844'
url_list = [f'http://openapi.seoul.go.kr:8088/{auth_key}/xml/CardSubwayStatsNew/1/999/{day}' for day in jan_list]
url_list

def get_jan_day_table(url):
    response = requests.get(url)
    df = pd.read_xml(response.text)
    return df

# 1월 1일부터 1월 28일까지 데이터 불러오기
df = pd.Series(url_list).map(get_jan_day_table)

# 데이터 병합
df = list(df)
df = pd.concat(df)

# 필요한 부분만 잘라내기
df = df.iloc[2:, 3:]
df.columns

# 인덱스 재설정
df = df.reset_index(drop=True)


df.shape #모양
df.dtypes #타입
df.columns # 컬럼

# 컬럼명 변경
df.columns = ['사용일자', '호선명', '역명', '승차총승객수', '하차총승객수', '등록일자']

second = df[df['호선명'] == '2호선'].groupby(['역명'])[['승차총승객수', '하차총승객수']].mean().reset_index() # 2호선만 역명 별 평균 승,하차 총 승객수
second.rename(columns = {'승차총승객수':'평균승차인원','하차총승객수':'평균하차인원'}, inplace = True) # inplace트루 안해주면 바뀐 열 이름이 적용되지 않음                   
second['역명'] = second['역명']+'역' # 무슨무슨'역 이라 붙여줌

In [2]:
'''새로운 파일 csv 불러오기'''
import pandas as pd
aaa = pd.read_csv('서울교통공사_1_8호선 역사 좌표(위경도) 정보_20211231.csv', encoding = 'CP949') # 좌표 쓰여있는 파일 불러옴
result = aaa[aaa['호선'] == 2] # 2호선만 남김
result = result.sort_values(by='역명') #역명으로 정렬
result = result[result['역명'] != '까치산'] #까치산이 있는 행 제거
# 데이터프레임 인덱스 1부터 시작하게 바꾸는 법
result.index = range(1, len(result)+1) #인덱스 1부터로 변경
second.index = range(1, len(second)+1) #얘도 인덱스 1부터로 변경
result = result[['위도', '경도']] # 위도, 경도만 남김
merged = pd.concat([second, result], axis=1)

In [4]:
'''지도에 그리기'''
# 시도 1
import folium
map = folium.Map(location = [37.529622, 126.984307], zoom_start=12)
for i in merged.index:
    folium.CircleMarker([merged['위도'][i], merged['경도'][i]], radius=merged['평균승차인원'][i]/2000,
                        popup = merged['역명'][i],color='#3186cc', fill_color='#3186cc').add_to(map)
map